In [63]:
import pandas as pd
import numpy as np
import glob as glob
import os


# main_dir = '/global/cfs/cdirs/metatlas/raw_data'
# main_dir = '/global/cfs/cdirs/metatlas/projects/carbon_network/raw_data/global/cfs/cdirs/metatlas/projects/rawdata_for_scn'
main_dir = '/pscratch/sd/b/bpb/massive'

In [64]:
# dates, buddy_mdm_files = get_files_from_disk('/pscratch/sd/b/bpb/metatlas_mdm_parquet_files', 'parquet-failed')
# len(buddy_mdm_files)

In [65]:
# dates, buddy_mdm_files = get_files_from_disk('/pscratch/sd/b/bpb/metatlas_mdm_parquet_files', 'parquet')
# len(buddy_mdm_files)

In [66]:
from subprocess import check_output
def get_files_from_disk(directory,extension):
    """
    Get on disk with date
    """
    get_with_date = ''.join(['find %s -iname "*%s"' % (directory,extension),' -printf "%Ts SplitThat%p\n"'])
    files = check_output(get_with_date, shell=True)
    files = files.decode('utf-8').splitlines()
    files = [f.split('SplitThat') for f in files]
    dates = [int(f[0].strip()) for f in files]
    files = [f[1].strip() for f in files]
    return dates,files

# hdf5_files = glob.glob(main_dir + '/**/*.h5', recursive=True)
dates, hdf5_files = get_files_from_disk(main_dir, 'h5')
dates, buddy_mdm_files = get_files_from_disk(main_dir, 'parquet')
dates, buddy_failed_mdm_files = get_files_from_disk(main_dir, 'parquet-failed')

# buddy_mdm_files = glob.glob(main_dir + '/**/*.parquet', recursive=True)
# buddy_failed_mdm_files = glob.glob(main_dir + '/**/*.parquet-failed', recursive=True)
df_h5 = pd.DataFrame({'h5': hdf5_files})
df_h5['no_extension'] = df_h5['h5'].apply(lambda x: x.replace('.h5', ''))
df_h5.set_index('no_extension', inplace=True)
df_buddy = pd.DataFrame({'buddy': buddy_mdm_files})
df_buddy['no_extension'] = df_buddy['buddy'].apply(lambda x: x.replace('.parquet', ''))
df_buddy.set_index('no_extension', inplace=True)
df_buddy_failed = pd.DataFrame({'buddy_failed': buddy_failed_mdm_files})
df_buddy_failed['no_extension'] = df_buddy_failed['buddy_failed'].apply(lambda x: x.replace('.parquet-failed', ''))
df_buddy_failed.set_index('no_extension', inplace=True)

df = df_h5.join(df_buddy, how='outer').join(df_buddy_failed, how='outer')
df




,h5,buddy,buddy_failed
no_extension,,,
/pscratch/sd/b/bpb/massive/v01/MSV000068104/datasets/68104/PAe000097/PAe000097_mzXML_200607122305/07160301c1_030716141811,/pscratch/sd/b/bpb/massive/v01/MSV000068104/da...,NaN,NaN
/pscratch/sd/b/bpb/massive/v01/MSV000068104/datasets/68104/PAe000097/PAe000097_mzXML_200607122305/07160302c1,/pscratch/sd/b/bpb/massive/v01/MSV000068104/da...,NaN,NaN
/pscratch/sd/b/bpb/massive/v01/MSV000068104/datasets/68104/PAe000097/PAe000097_mzXML_200607122305/07160303c1,/pscratch/sd/b/bpb/massive/v01/MSV000068104/da...,NaN,NaN
/pscratch/sd/b/bpb/massive/v01/MSV000068104/datasets/68104/PAe000097/PAe000097_mzXML_200607122305/07160304c1,/pscratch/sd/b/bpb/massive/v01/MSV000068104/da...,NaN,NaN
/pscratch/sd/b/bpb/massive/v01/MSV000068104/datasets/68104/PAe000097/PAe000097_mzXML_200607122305/07160305c1,/pscratch/sd/b/bpb/massive/v01/MSV000068104/da...,NaN,NaN
...,...,...,...
/pscratch/sd/b/bpb/massive/z01/MSV000092338/ccms_peak/Blank_1,/pscratch/sd/b/bpb/massive/z01/MSV000092338/cc...,NaN,NaN
/pscratch/sd/b/bpb/massive/z01/MSV000092338/ccms_peak/COPRO_I,/pscratch/sd/b/bpb/massive/z01/MSV000092338/cc...,NaN,NaN
/pscratch/sd/b/bpb/massive/z01/MSV000092338/ccms_peak/COPRO_II,/pscratch/sd/b/bpb/massive/z01/MSV000092338/cc...,NaN,NaN


In [67]:
len(buddy_failed_mdm_files)

2690

In [71]:
buddy_failed_mdm_files[1000]

'/pscratch/sd/b/bpb/massive/v01/MSV000092394/updates/2023-07-12_luciacancelada_fa458d93/peak/neg_peak/blank_std_6_neg.parquet-failed'

In [33]:
import h5py
import multiprocessing


def good_hdf(file_path):
    try:
        with h5py.File(file_path, 'r') as file:
            if len([key for key in file.keys() if key.startswith('ms2')]) > 0:
                num_pos_spectra = file['ms2_pos'].shape[0]
                num_neg_spectra = file['ms2_neg'].shape[0]
                return [num_pos_spectra, num_neg_spectra]
            else:
                return [0, 0]
    except Exception as e:
        return None

def parallel_good_hdf(file_paths):
    pool = multiprocessing.Pool(20)
    results = pool.map(good_hdf, file_paths)
    pool.close()
    pool.join()
    return results

out = parallel_good_hdf(df['h5'].tolist())
df['spectra_check'] = out
df.to_csv('metatlas_files_hdf5_checked.csv')

In [43]:
spectra = pd.read_csv('metatlas_files_hdf5_checked.csv')

def get_counts(x,position):
    # x = eval(x)
    if type(x) == float:
        return np.nan
    else:
        x = eval(x)
    if len(x) ==2:
        return x[position]
    else:
        return np.nan

spectra['ms2_pos'] = spectra['spectra_check'].apply(lambda x: get_counts(x,0))
spectra['ms2_neg'] = spectra['spectra_check'].apply(lambda x: get_counts(x,1))
cols = ['h5','ms2_pos','ms2_neg']
spectra = spectra[pd.notna(spectra['ms2_neg'])]
spectra = spectra[spectra['ms2_neg']>0]
print(df.shape)
df = pd.merge(df,spectra[cols],left_on='h5',right_on='h5',how='inner')
# df['buddy_failed'] = np.nan
# df = df[pd.isna(df['buddy'])]
# spectra
# spectra
df

(312592, 3)


,h5,buddy,buddy_failed,ms2_pos,ms2_neg
0,/pscratch/sd/b/bpb/massive/v01/MSV000079542/cc...,/pscratch/sd/b/bpb/massive/v01/MSV000079542/cc...,NaN,0.0,265264.0
1,/pscratch/sd/b/bpb/massive/v01/MSV000079542/cc...,/pscratch/sd/b/bpb/massive/v01/MSV000079542/cc...,NaN,0.0,272060.0
2,/pscratch/sd/b/bpb/massive/v01/MSV000079542/cc...,/pscratch/sd/b/bpb/massive/v01/MSV000079542/cc...,NaN,0.0,231681.0
3,/pscratch/sd/b/bpb/massive/v01/MSV000079542/cc...,/pscratch/sd/b/bpb/massive/v01/MSV000079542/cc...,NaN,0.0,253839.0
4,/pscratch/sd/b/bpb/massive/v01/MSV000079542/cc...,/pscratch/sd/b/bpb/massive/v01/MSV000079542/cc...,NaN,0.0,265968.0
...,...,...,...,...,...
27006,/pscratch/sd/b/bpb/massive/z01/MSV000092114/cc...,/pscratch/sd/b/bpb/massive/z01/MSV000092114/cc...,NaN,0.0,155075.0
27007,/pscratch/sd/b/bpb/massive/z01/MSV000092114/cc...,/pscratch/sd/b/bpb/massive/z01/MSV000092114/cc...,NaN,0.0,4170.0
27008,/pscratch/sd/b/bpb/massive/z01/MSV000092114/cc...,/pscratch/sd/b/bpb/massive/z01/MSV000092114/cc...,NaN,0.0,213481.0
27009,/pscratch/sd/b/bpb/massive/z01/MSV000092114/cc...,/pscratch/sd/b/bpb/massive/z01/MSV000092114/cc...,NaN,0.0,14360.0


In [68]:
for c in df.columns:
    print(c, df[c].count(),df.shape[0])

# mzml 27011 27011
# mzxml 0 27011
# h5 27011 27011
# buddy 23204 27011
# buddy_failed 2974 27011
# no_extension 27011 27011
# ms2_pos 27011 27011
# ms2_neg 27011 27011


h5 312592 312592
buddy 24321 312592
buddy_failed 2690 312592


In [7]:
df = spectra[pd.notna(spectra['h5'])].copy()
df.reset_index(inplace=True,drop=False)
df.shape

(480, 6)

In [45]:

df = df.sample(frac=1).reset_index(drop=True)


In [25]:
df.shape

(206, 4)

/tmp/ipykernel_298622/1287852194.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['spectra_check'] = out


In [17]:
20e3/60/60

5.555555555555555

In [39]:
df[pd.notna(df['buddy_failed'])].iloc[0]['buddy_failed']

'/pscratch/sd/b/bpb/massive/v06/MSV000092394/updates/2023-07-12_luciacancelada_fa458d93/peak/neg_peak/JCB-IP_5Jan23_21_Dark_neg_2.parquet-failed'

In [62]:
idx1 = pd.isna(df['buddy'])
idx2 = pd.notna(df['h5'])
idx3 = pd.isna(df['buddy_failed'])
df = df[(idx1) & (idx2) & (idx3)]
# # df[(idx1) & (idx2) & (idx3)]
df

,h5,buddy,buddy_failed
no_extension,,,


In [20]:
df = df[pd.notna(df['buddy'])]

In [47]:
df = df.sample(frac=1).reset_index(drop=True)
df.shape

(3452, 5)

In [43]:
5002/200

25.01

In [48]:
job_dir = '/pscratch/sd/b/bpb/metatlas_mdm_parquet_files'
my_bin = '/global/common/software/m2650/msbuddy/bin/python'
my_py = '/global/homes/b/bpb/repos/carbon_network/carbon_network/build/do_mdm_buddy_on_h5.py'
# idx = (df['h5_completed'] == True) & (df['buddy_completed'] == False) & (df['buddy_failed'] == False)# & (df['h5'].str.contains('neg',case=False))
# print(sum(idx))
job_counter = 0
counter = 0
job_files = []
for file in df['h5'].unique():
    if job_counter > 200:
        job_counter = 0
        counter += 1
        print(counter)
    if job_counter ==0:
        job_file = os.path.join(job_dir, 'job_%d.sbatch'%counter)
        with open(job_file, 'w') as f:
            job_files.append(job_file)
            f.write('#!/bin/bash\n')
            f.write('#SBATCH --job-name="%s"\n'%os.path.basename(file).replace('.h5', ''))
            f.write('#SBATCH --output="%s"\n'%os.path.join(job_dir, os.path.basename(file).replace('.h5', '.out')))
            f.write('#SBATCH --error="%s"\n'%os.path.join(job_dir, os.path.basename(file).replace('.h5', '.err')))
            f.write('#SBATCH --time=08:00:00\n')
            f.write('#SBATCH --qos=regular\n')
            f.write('#SBATCH --ntasks-per-node=128\n')
            f.write('#SBATCH --licenses=cfs\n')
            f.write('#SBATCH --exclusive\n')
            f.write('#SBATCH --constraint=cpu\n')
            f.write('#SBATCH --nodes=1\n')
            f.write('#SBATCH --cpus-per-task=1\n')
            f.write('#SBATCH --account=m2650\n\n\n')
    
    with open(job_file, 'a') as f:
        f.write('%s %s --do_buddy --file "%s"\n'%(my_bin, my_py, file))
        # print(job_counter)
        job_counter += 1
    # if counter > 3:
        # break
        # f.write('#SBATCH --mail-type=END\n')
        # f.write('#SBATCH --mail-user=%s\n'%os.environ['USER'])
with open(os.path.join(job_dir,'submit_all_jobs.sh'),'w') as fid:
    for job_file in job_files:
        fid.write('sbatch "%s"\n'%job_file)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [13]:
sum(df['h5'].str.contains('BIOD'))

206

In [12]:
for i,row in df.iterrows():
    print(os.path.basename(row['h5']))
    

20230403_EB_BGS_107002-011_BIODESERT_Metagenome_EXP120A_C18-EP_USDAY72349_NEG_MS2_19_ARG-Quiroga-SanNicolas-1PLH-2GZM_1__638.h5
20220321_JGI-TH_PA_507784_Frtlzr_SoilSD_IDX_C18_USDAY59443_NEG_MSMS_9-1_CORN-21-ABG-SD-01FS-411-2_1_Rg80to1200-CE102040-soil-S1_Run30.h5
20210920_JGI-AK-TH_NB_504882_SoilWetDry_final_QE-HF_C18_USDAY63680_NEG_MSMS_41_Mesocosm-control_1_Rg80to1200-CE102040-soil-Ultisol-S1_Run29.h5
20231018_EB_MdR_109570-002_WAVEstab_20231017_EXP120A_C18-EP_USDAY72349_NEG_MS2_10_supern-WAVE-NatCom-Salts-Day7_2_3uL_090.h5
20230403_EB_BGS_107002-011_BIODESERT_Metagenome_EXP120A_C18-EP_USDAY72349_NEG_MS2_217_ESP-Pueyo-ZaragozaArido-1PLH-2GZM_1__301.h5
20231018_EB_MdR_109570-002_WAVEstab_20231017_EXP120A_C18-EP_USDAY72349_NEG_MS2_5_supern-WAVE-NatCom-NLDM-Day0_1_6uL_102.h5
20230403_EB_BGS_107002-011_BIODESERT_Metagenome_EXP120A_C18-EP_USDAY72349_NEG_MS2_141_NAM-Blaum-Sandveld-2PLM-2GZM_1__334.h5
20221110_EB_MdR_101544-059_HumicAcid_20221110_EXP120A_C18-EP_USDAY63672_NEG_MS2_15_HumicA

In [44]:
df = pd.read_hdf('/pscratch/sd/b/bpb/massive/v01/MSV000082048/peak/Plate_25/Wash_02.h5', 'ms2_pos')
df

,mz,i,rt,polarity,precursor_MZ,precursor_intensity,collision_energy
0,53.379326,803.464233,0.039861,1,118.086243,2.284760e+06,30.0
1,54.484489,969.328491,0.039861,1,118.086243,2.284760e+06,30.0
2,55.934872,11133.907227,0.039861,1,118.086243,2.284760e+06,30.0
3,58.065578,147494.937500,0.039861,1,118.086243,2.284760e+06,30.0
4,59.073410,191423.109375,0.039861,1,118.086243,2.284760e+06,30.0
...,...,...,...,...,...,...,...
44392,164.908264,986.529297,15.985592,1,187.126373,6.275787e+04,30.0
44393,176.938873,1009.572815,15.985592,1,187.126373,6.275787e+04,30.0
44394,178.300995,900.186707,15.985592,1,187.126373,6.275787e+04,30.0
44395,187.126389,2045.288208,15.985592,1,187.126373,6.275787e+04,30.0


In [51]:
p = pd.read_parquet('/pscratch/sd/b/bpb/massive/v01/MSV000088756/ccms_peak/NEG/CuF3_neg.parquet')
p

,precursor_mz,rt,filename,coisolated_precursor_count,ms2_mz_vals,ms2_i_vals,predicted_formula,estimated_fdr
0,341.066833,2.853,/pscratch/sd/b/bpb/massive/v01/MSV000088756/cc...,1,"[253.05072021484375, 254.05833435058594, 263.7...","[966332.25, 653806.375, 48854.87890625, 932489...",C18H14O7,2.954410e-05
1,341.030487,2.014,/pscratch/sd/b/bpb/massive/v01/MSV000088756/cc...,1,"[250.7098846435547, 297.0406494140625, 297.004...","[15491.8330078125, 1740046.625, 28648.53125, 1...",C17H10O8,8.473101e-06
2,327.051147,2.068,/pscratch/sd/b/bpb/massive/v01/MSV000088756/cc...,1,"[212.0481414794922, 237.05482482910156, 238.06...","[25043.509765625, 12650.857421875, 11585.83105...",C17H12O7,9.156523e-03
3,343.046143,2.215,/pscratch/sd/b/bpb/massive/v01/MSV000088756/cc...,1,"[240.04356384277344, 253.0492706298828, 255.98...","[5001.92431640625, 6284.67431640625, 15270.579...",C17H12O8,1.898955e-08
4,357.062042,3.075,/pscratch/sd/b/bpb/massive/v01/MSV000088756/cc...,1,"[269.04583740234375, 269.0745849609375, 270.05...","[306555.25, 18905.13671875, 192408.09375, 1249...",C18H14O8,2.547071e-08
5,187.097733,1.805,/pscratch/sd/b/bpb/massive/v01/MSV000088756/cc...,1,"[123.0814437866211, 187.0977020263672, 169.087...","[18018.21875, 146837.9375, 6814.76220703125, 3...",C9H16O4,0.000000e+00
6,269.045793,3.788,/pscratch/sd/b/bpb/massive/v01/MSV000088756/cc...,2,"[181.0661163330078, 197.06103515625, 225.05599...","[16393.73046875, 18737.615234375, 215383.1875,...",C15H10O5,2.995318e-07
7,206.972885,8.515,/pscratch/sd/b/bpb/massive/v01/MSV000088756/cc...,1,"[206.9733428955078, 162.98291015625, 115.48523...","[12534.1796875, 77767.3671875, 2574.9375, 7198...",C5H8N2OS3,0.000000e+00
8,259.118805,1.651,/pscratch/sd/b/bpb/massive/v01/MSV000088756/cc...,1,"[183.10296630859375, 169.12364196777344, 171.1...","[9157.1982421875, 56413.1796875, 83467.9375, 6...",C12H20O6,1.906624e-06
9,357.062012,3.114,/pscratch/sd/b/bpb/massive/v01/MSV000088756/cc...,1,"[269.0458068847656, 270.0538024902344, 280.037...","[102817.4765625, 65716.7890625, 50513.00390625...",C18H14O8,1.750300e-07
